In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/widsdatathon2020/training_v2.csv
/kaggle/input/widsdatathon2020/samplesubmission.csv
/kaggle/input/widsdatathon2020/unlabeled.csv
/kaggle/input/widsdatathon2020/WiDS Datathon 2020 Dictionary.csv
/kaggle/input/widsdatathon2020/solution_template.csv


In [2]:
!pip install imblearn

     |████████████████████████████████| 7.1MB 3.0MB/s eta 0:00:01
ERROR: allennlp 0.9.0 requires flaky, which is not installed.
ERROR: allennlp 0.9.0 requires responses>=0.7, which is not installed.
ERROR: kmeans-smote 0.1.2 has requirement imbalanced-learn<0.5,>=0.4.0, but you'll have imbalanced-learn 0.6.1 which is incompatible.
ERROR: kmeans-smote 0.1.2 has requirement numpy<1.16,>=1.13, but you'll have numpy 1.18.1 which is incompatible.
ERROR: kmeans-smote 0.1.2 has requirement scikit-learn<0.21,>=0.19.0, but you'll have scikit-learn 0.22.1 which is incompatible.
ERROR: hypertools 0.6.2 has requirement scikit-learn<0.22,>=0.19.1, but you'll have scikit-learn 0.22.1 which is incompatible.
ERROR: allennlp 0.9.0 has requirement spacy<2.2,>=2.1.0, but you'll have spacy 2.2.3 which is incompatible.
  Found existing installation: scikit-learn 0.21.3
    Uninstalling scikit-learn-0.21.3:
      Successfully uninstalled scikit-learn-0.21.3


In [3]:
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

Using TensorFlow backend.


In [4]:
train= pd.read_csv("/kaggle/input/widsdatathon2020/training_v2.csv", header=0)
test = pd.read_csv("/kaggle/input/widsdatathon2020/unlabeled.csv")

In [5]:
train_categorical_cols = []
train_numerical_cols = []
for c in  train.columns:
    if(train[c].dtype=='object'):
        train_categorical_cols.append(c)
    else:
        train_numerical_cols.append(c)
test_categorical_cols = []
test_numerical_cols = []
for c in  test.columns:
    if(test[c].dtype=='object'):
        test_categorical_cols.append(c)
    else:
        test_numerical_cols.append(c)

print(train_categorical_cols)
print(len(train_categorical_cols))
print(len(train_numerical_cols))
print(test_categorical_cols)
print(len(test_categorical_cols))
print(len(test_numerical_cols))

['ethnicity', 'gender', 'hospital_admit_source', 'icu_admit_source', 'icu_stay_type', 'icu_type', 'apache_3j_bodysystem', 'apache_2_bodysystem']
8
178
['ethnicity', 'gender', 'hospital_admit_source', 'icu_admit_source', 'icu_stay_type', 'icu_type', 'apache_3j_bodysystem', 'apache_2_bodysystem']
8
178


In [6]:
train[train_categorical_cols] = train[train_categorical_cols].fillna("")
test[test_categorical_cols] = test[test_categorical_cols].fillna("")

In [7]:
train.fillna(train.mean(), inplace=True)
test.fillna(test.mean(), inplace=True)


In [8]:
train_numerical_cols.remove('encounter_id')
train_numerical_cols.remove('patient_id')
train_numerical_cols.remove('hospital_id')
train_numerical_cols.remove('hospital_death')

In [9]:
std_scale = StandardScaler().fit(train[train_numerical_cols])
train[train_numerical_cols] = std_scale.transform(train[train_numerical_cols])

In [10]:
train_allcols = train.columns.values.tolist()
test_allcols = test.columns.values.tolist()

In [11]:
allcols = train.columns.values.tolist()


In [12]:
inputcols = allcols[4:]
inputcols.remove('readmission_status')

In [13]:
outputcols = allcols[3]


In [14]:
X = train[inputcols]
y = train[outputcols]

In [19]:
label = LabelEncoder()
for c in  X.columns:
    if(X[c].dtype == 'object'):
        X[c]=label.fit_transform(X[c])
    else:
        X[c]=X[c]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [17]:
from collections import Counter
counter = Counter(y)
print(counter)

Counter({0: 83798, 1: 7915})


In [20]:
over = SMOTE(sampling_strategy=0.1)
under = RandomUnderSampler(sampling_strategy=0.5)
steps = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps)
# transform the dataset
X, y = pipeline.fit_resample(X, y)
# summarize the new class distribution
counter = Counter(y)
print(counter)

Counter({0: 16758, 1: 8379})


In [21]:
X.head()

,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,...,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem
0,-0.140963,-0.682241,-0.474441,3,2,0.052108,0,1,1.605506,2,...,-0.029405,-0.12676,-0.541226,-0.115168,-0.164559,-0.084689,-0.064666,-0.145734,2,2
1,-1.117538,1.604269,-0.474441,3,1,-0.657078,0,1,0.780137,0,...,-0.029405,-0.12676,-0.541226,-0.115168,-0.164559,-0.084689,-0.064666,-0.145734,8,5
2,0.713540,0.385247,-0.474441,6,1,0.528008,3,1,0.033374,0,...,-0.029405,-0.12676,1.862175,-0.115168,-0.164559,-0.084689,-0.064666,-0.145734,9,7
3,0.286288,-0.874127,-0.474441,3,1,-1.375595,0,1,0.836908,0,...,-0.029405,-0.12676,-0.541226,-0.115168,-0.164559,-0.084689,-0.064666,-0.145734,1,1
4,-1.666862,0.344562,-0.474441,0,1,-0.115857,4,1,1.264878,0,...,-0.029405,-0.12676,-0.541226,-0.115168,-0.164559,-0.084689,-0.064666,-0.145734,6,4


In [23]:
y.unique()

array([0, 1])

In [24]:
model = DecisionTreeClassifier()
model.fit(X,y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [26]:
label = LabelEncoder()
x_test = test[inputcols]
for c in  x_test.columns:
    if(x_test[c].dtype == 'object'):
        x_test[c]=label.fit_transform(x_test[c])
    else:
        x_test[c]=x_test[c]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [27]:
test_output_y = model.predict(x_test)

In [28]:
len(test_output_y)

array([0, 0, 0, ..., 0, 0, 0])

In [29]:
test["hospital_death"] = test_output_y
test[["encounter_id","hospital_death"]].to_csv("submission_smote1.csv",index=False)